In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
import numpy as np
import random
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score, average_precision_score, matthews_corrcoef, balanced_accuracy_score
import pandas as pd
import seaborn as sns

# Set random seeds for reproducibility
def set_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed()

# Define the device for computation
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Mount Google Drive to save and load the model (if using Google Colab)
try:
    from google.colab import drive
    drive.mount('/content/drive')
    base_path = '/content/drive/My Drive/FYP'
except ImportError:
    base_path = '.'  # Use current directory if not in Colab

# Define the folder to save model checkpoints
date_str = datetime.now().strftime('%Y%m%d')
checkpoint_folder = f'{base_path}/VGGModel/HQ3latest_{date_str}/_valloss'
os.makedirs(checkpoint_folder, exist_ok=True)

# Data Augmentation for Training Set
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),  # VGG expects 224x224 input size
    transforms.RandomHorizontalFlip(p=0.5),  # Flip left/right
    transforms.RandomVerticalFlip(p=0.3),  # Flip up/down
    transforms.RandomRotation(degrees=10),  # Small rotation
    transforms.RandomAffine(degrees=0, translate=(0.05, 0.05)),  # Slight shifts
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),  # Small color changes
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

# Simple resizing for validation and test sets
transform_val_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

# Load datasets
data_path = f'{base_path}/Dataset/HQ3/train'
test_data_path = f'{base_path}/Dataset/HQ3/test'
train_dataset = datasets.ImageFolder(data_path, transform=transform_train)
test_dataset = datasets.ImageFolder(test_data_path, transform=transform_val_test)

# Split the dataset into 70% training and 30% validation
train_size = int(0.85 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

# Set validation set transformation explicitly
val_dataset.dataset.transform = transform_val_test

# Create data loaders
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

# Use Pre-trained VGG-16 model and modify it for binary classification
class VGG16Modified(nn.Module):
    def __init__(self, num_classes=2, dropout_rate=0.4):
        super(VGG16Modified, self).__init__()
        from torchvision.models import vgg16, VGG16_Weights

        # Load pre-trained VGG16 model
        self.vgg = vgg16(weights=VGG16_Weights.IMAGENET1K_V1)

        # Replace the classifier with a custom classification layer
        num_ftrs = self.vgg.classifier[6].in_features
        self.vgg.classifier[6] = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(num_ftrs, num_classes)
        )

    def forward(self, x):
        return self.vgg(x)

# Initialize the VGG model
model = VGG16Modified().to(device)

# Define loss function and optimizer with L2 regularization (weight decay)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.0001, weight_decay=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=3)

def calculate_metrics(y_true, y_pred, y_scores):
    """Calculate various classification metrics."""
    metrics = {}
    metrics['accuracy'] = accuracy_score(y_true, y_pred) * 100
    metrics['balanced_accuracy'] = balanced_accuracy_score(y_true, y_pred)
    metrics['precision'] = precision_score(y_true, y_pred, average='binary')
    metrics['recall'] = recall_score(y_true, y_pred, average='binary')
    metrics['f1'] = f1_score(y_true, y_pred, average='binary')
    metrics['mcc'] = matthews_corrcoef(y_true, y_pred)

    # For ROC-AUC and PR-AUC, we need probabilities
    # Ensure y_scores are probabilities of the positive class
    metrics['roc_auc'] = roc_auc_score(y_true, y_scores)
    metrics['pr_auc'] = average_precision_score(y_true, y_scores)

    return metrics

def evaluate(model, loader, criterion):
    """Evaluate model and return predictions, true labels, losses and metrics."""
    model.eval()
    all_preds = []
    all_labels = []
    all_scores = []
    total_loss = 0.0

    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            # Get predicted class and probability scores
            probs = torch.softmax(outputs, dim=1)
            pos_probs = probs[:, 1].cpu().numpy()  # Probability of positive class
            _, predicted = torch.max(outputs.data, 1)

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_scores.extend(pos_probs)

    # Calculate metrics
    metrics = calculate_metrics(all_labels, all_preds, all_scores)
    metrics['loss'] = total_loss / len(loader)

    return metrics, all_preds, all_labels, all_scores

def train(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=30):
    """Train the model and track metrics with early stopping based on validation loss."""
    # Initialize lists to store metrics
    metrics_df = pd.DataFrame(columns=['epoch', 'training_loss', 'val_loss', 'val_accuracy',
                                       'val_balanced_accuracy', 'val_precision', 'val_recall',
                                       'val_f1', 'val_mcc', 'val_roc_auc', 'val_pr_auc'])

    best_val_acc = 0.0
    best_val_loss = float('inf')  # Initialize best validation loss
    checkpoint_model_path = os.path.join(checkpoint_folder, f"checkpoint_model_vgg_{date_str}.pth")
    best_model_path = os.path.join(checkpoint_folder, f"best_model_vgg_{date_str}.pth")
    early_stopping_path = os.path.join(checkpoint_folder, f"early_stopping_model_vgg_{date_str}.pth")

    # Early stopping parameters
    patience = 5  # Number of epochs to wait for improvement
    counter = 0
    early_stop = False

    for epoch in range(epochs):
        if early_stop:
            print(f"Early stopping triggered after {epoch} epochs")
            break

        # Training phase
        model.train()
        running_loss = 0.0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()

            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=2.0)
            optimizer.step()

            running_loss += loss.item()

        # Calculate training loss
        train_loss = running_loss / len(train_loader)

        # Validation phase
        val_metrics, _, _, _ = evaluate(model, val_loader, criterion)
        val_loss = val_metrics['loss']
        val_acc = val_metrics['accuracy']

        # Update learning rate scheduler
        scheduler.step(val_acc)

        # Save checkpoint
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'train_loss': train_loss,
            'val_loss': val_loss,
            'val_acc': val_acc
        }, checkpoint_model_path)

        # Save best model based on validation accuracy
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), best_model_path)
            print(f"New best model saved with validation accuracy: {val_acc:.2f}%")

        # Early stopping check based on validation loss
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            counter = 0
            # Save the model with the best validation loss
            torch.save(model.state_dict(), early_stopping_path)
            print(f"New best validation loss: {val_loss:.6f}")
        else:
            counter += 1
            print(f"EarlyStopping counter: {counter} out of {patience}")
            if counter >= patience:
                early_stop = True
                print(f"Early stopping triggered: validation loss didn't improve for {patience} epochs")

        # Print epoch results
        print(f"Epoch [{epoch+1}/{epochs}] - "
              f"Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}, "
              f"Val Acc: {val_acc:.2f}%, "
              f"Val F1: {val_metrics['f1']:.4f}, "
              f"Val ROC-AUC: {val_metrics['roc_auc']:.4f}")

        # Add metrics to dataframe
        metrics_df = pd.concat([metrics_df, pd.DataFrame({
            'epoch': [epoch + 1],
            'training_loss': [train_loss],
            'val_loss': [val_loss],
            'val_accuracy': [val_metrics['accuracy']],
            'val_balanced_accuracy': [val_metrics['balanced_accuracy']],
            'val_precision': [val_metrics['precision']],
            'val_recall': [val_metrics['recall']],
            'val_f1': [val_metrics['f1']],
            'val_mcc': [val_metrics['mcc']],
            'val_roc_auc': [val_metrics['roc_auc']],
            'val_pr_auc': [val_metrics['pr_auc']]
        })], ignore_index=True)

    # Early stopping information
    if early_stop:
        print(f"Training stopped early at epoch {epoch}")
        early_stopping_info = pd.DataFrame({
            'stopped_epoch': [epoch],
            'best_val_loss': [best_val_loss],
            'patience': [patience]
        })
        early_stopping_info.to_csv(os.path.join(checkpoint_folder, f"early_stopping_info_{date_str}.csv"), index=False)

    # Save metrics to CSV
    metrics_csv_path = os.path.join(checkpoint_folder, f"metrics_{date_str}.csv")
    metrics_df.to_csv(metrics_csv_path, index=False)
    print(f"Training metrics saved to {metrics_csv_path}")

    # Plot and save training curves
    plot_training_curves(metrics_df, checkpoint_folder)

    # Plot confusion matrix for best model (both accuracy-based and loss-based)
    print("\nEvaluating best accuracy model:")
    model.load_state_dict(torch.load(best_model_path))
    val_metrics_acc, val_preds_acc, val_labels_acc, _ = evaluate(model, val_loader, criterion)
    plot_confusion_matrix(val_labels_acc, val_preds_acc, ['Normal', 'Tumor'], checkpoint_folder, "validation_best_acc")

    print("\nEvaluating early stopping model (best validation loss):")
    model.load_state_dict(torch.load(early_stopping_path))
    val_metrics_loss, val_preds_loss, val_labels_loss, _ = evaluate(model, val_loader, criterion)
    plot_confusion_matrix(val_labels_loss, val_preds_loss, ['Normal', 'Tumor'], checkpoint_folder, "validation_early_stop")

    # Compare the models
    print(f"\nComparison - Best Accuracy Model vs Early Stopping Model:")
    print(f"Best Accuracy Model - Val Acc: {val_metrics_acc['accuracy']:.2f}%, Val Loss: {val_metrics_acc['loss']:.6f}")
    print(f"Early Stopping Model - Val Acc: {val_metrics_loss['accuracy']:.2f}%, Val Loss: {val_metrics_loss['loss']:.6f}")

    return metrics_df, early_stop, epoch

def plot_training_curves(metrics_df, save_folder):
    """Plot and save training curves."""
    plt.figure(figsize=(20, 15))

    # Plot 1: Training vs Validation Loss
    plt.subplot(2, 2, 1)
    plt.plot(metrics_df['epoch'], metrics_df['training_loss'], 'b-', label='Training Loss')
    plt.plot(metrics_df['epoch'], metrics_df['val_loss'], 'r-', label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training vs Validation Loss')
    plt.legend()
    plt.grid(True)

    # Plot 2: Validation Accuracy
    plt.subplot(2, 2, 2)
    plt.plot(metrics_df['epoch'], metrics_df['val_accuracy'], 'g-')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy (%)')
    plt.title('Validation Accuracy')
    plt.grid(True)

    # Plot 3: Validation F1, Precision, Recall
    plt.subplot(2, 2, 3)
    plt.plot(metrics_df['epoch'], metrics_df['val_f1'], 'c-', label='F1 Score')
    plt.plot(metrics_df['epoch'], metrics_df['val_precision'], 'm-', label='Precision')
    plt.plot(metrics_df['epoch'], metrics_df['val_recall'], 'y-', label='Recall')
    plt.xlabel('Epochs')
    plt.ylabel('Score')
    plt.title('Validation F1, Precision, Recall')
    plt.legend()
    plt.grid(True)

    # Plot 4: Validation ROC-AUC and PR-AUC
    plt.subplot(2, 2, 4)
    plt.plot(metrics_df['epoch'], metrics_df['val_roc_auc'], 'b-', label='ROC-AUC')
    plt.plot(metrics_df['epoch'], metrics_df['val_pr_auc'], 'r-', label='PR-AUC')
    plt.plot(metrics_df['epoch'], metrics_df['val_mcc'], 'g-', label='MCC')
    plt.xlabel('Epochs')
    plt.ylabel('Score')
    plt.title('Validation ROC-AUC, PR-AUC, and MCC')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.savefig(os.path.join(save_folder, f"training_curves_{date_str}.png"), dpi=300)
    plt.show()

def plot_confusion_matrix(y_true, y_pred, class_names, save_folder, dataset_name):
    """Plot and save confusion matrix."""
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title(f'Confusion Matrix - {dataset_name.capitalize()} Set')
    plt.tight_layout()
    plt.savefig(os.path.join(save_folder, f"confusion_matrix_{dataset_name}_{date_str}.png"), dpi=300)
    plt.show()

def test_model(model, test_loader, criterion, model_name="best"):
    """Evaluate the model on the test set and visualize results."""
    print(f"\nEvaluating {model_name} model on test set...")
    test_metrics, test_preds, test_labels, test_scores = evaluate(model, test_loader, criterion)

    # Print test metrics
    print(f"\nTest Set Metrics ({model_name} model):")
    print(f"Loss: {test_metrics['loss']:.6f}")
    print(f"Accuracy: {test_metrics['accuracy']:.2f}%")
    print(f"Balanced Accuracy: {test_metrics['balanced_accuracy']:.4f}")
    print(f"Precision: {test_metrics['precision']:.4f}")
    print(f"Recall: {test_metrics['recall']:.4f}")
    print(f"F1 Score: {test_metrics['f1']:.4f}")
    print(f"MCC: {test_metrics['mcc']:.4f}")
    print(f"ROC-AUC: {test_metrics['roc_auc']:.4f}")
    print(f"PR-AUC: {test_metrics['pr_auc']:.4f}")

    # Plot confusion matrix for test set
    plot_confusion_matrix(test_labels, test_preds, ['Normal', 'Tumor'], checkpoint_folder, f"test_{model_name}")

    # Save test metrics to CSV
    test_metrics_df = pd.DataFrame({metric: [value] for metric, value in test_metrics.items()})
    test_metrics_df.to_csv(os.path.join(checkpoint_folder, f"test_metrics_{model_name}_{date_str}.csv"), index=False)

    return test_metrics

# ------------------- Main Training & Testing Routine -------------------
if __name__ == "__main__":
    # Train the model and collect metrics
    metrics_df, early_stopped, stopped_epoch = train(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=30)

    # Load best accuracy model for testing
    best_model_path = os.path.join(checkpoint_folder, f"best_model_vgg_{date_str}.pth")
    model.load_state_dict(torch.load(best_model_path))
    test_metrics_acc = test_model(model, test_loader, criterion, "best_accuracy")

    # Load early stopping model for testing (if different from best accuracy model)
    early_stopping_path = os.path.join(checkpoint_folder, f"early_stopping_model_vgg_{date_str}.pth")
    model.load_state_dict(torch.load(early_stopping_path))
    test_metrics_es = test_model(model, test_loader, criterion, "early_stopping")

    # Compare test results between models
    print("\nTest Set Comparison - Best Accuracy Model vs Early Stopping Model:")
    print(f"Best Accuracy Model - Test Acc: {test_metrics_acc['accuracy']:.2f}%, Test Loss: {test_metrics_acc['loss']:.6f}")
    print(f"Early Stopping Model - Test Acc: {test_metrics_es['accuracy']:.2f}%, Test Loss: {test_metrics_es['loss']:.6f}")

    if early_stopped:
        print(f"\nEarly stopping was triggered after {stopped_epoch} epochs")
    else:
        print("\nTraining completed without early stopping")

    print(f"\nTraining and evaluation complete. Results saved to {checkpoint_folder}")

Using device: cpu
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


New best model saved with validation accuracy: 95.61%
New best validation loss: 0.081745
Epoch [1/30] - Train Loss: 0.331784, Val Loss: 0.081745, Val Acc: 95.61%, Val F1: 0.9741, Val ROC-AUC: 0.9933


<ipython-input-3-995e747a950a>:247: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  metrics_df = pd.concat([metrics_df, pd.DataFrame({
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


EarlyStopping counter: 1 out of 5
Epoch [2/30] - Train Loss: 0.116234, Val Loss: 0.567935, Val Acc: 95.61%, Val F1: 0.9749, Val ROC-AUC: 0.9406


...

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


EarlyStopping counter: 4 out of 5
Epoch [5/30] - Train Loss: 0.057150, Val Loss: 0.102353, Val Acc: 95.61%, Val F1: 0.9749, Val ROC-AUC: 0.9982


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


New best model saved with validation accuracy: 99.12%
New best validation loss: 0.018944
Epoch [6/30] - Train Loss: 0.099987, Val Loss: 0.018944, Val Acc: 99.12%, Val F1: 0.9949, Val ROC-AUC: 0.9994


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


In [ ]:
!pip uninstall sympy
!pip install sympy --upgrade
!pip install torch torchvision --upgrade

Found existing installation: sympy 1.13.1
Uninstalling sympy-1.13.1:
  Would remove:
    /usr/local/bin/isympy
    /usr/local/lib/python3.11/dist-packages/isympy.py
    /usr/local/lib/python3.11/dist-packages/sympy-1.13.1.dist-info/*
    /usr/local/lib/python3.11/dist-packages/sympy/*
    /usr/local/share/man/man1/isympy.1
Proceed (Y/n)? y
  Successfully uninstalled sympy-1.13.1
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
Using cached sympy-1.14.0-py3-none-any.whl (6.3 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-